In [ ]:
!pip install numpy
!pip install matplotlib
!pip install elevation
!pip install richdem
!pip install -U -q PyDrive
!pip install rasterio

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import elevation
import richdem as rd
import rasterio

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8, 5.5)

In [ ]:
#download tiff file from google drive
downloaded = drive.CreateFile({'id':"14nfAg677VSg-zmerx1wNdAgNvEvIa-gl"})
downloaded.GetContentFile('map.tiff')  

In [ ]:
#dem_path = os.path.join(os.getcwd(), 'Shasta-30m-DEM.tif')
dem_path = os.path.join(os.getcwd(), 'map.tif')

In [ ]:
elevation.clip(bounds=(6.70, 80.38, 7.59, 81.23), output=dem_path)

In [ ]:
matrix = rasterio.open(dem_path).read()[0]

In [ ]:
matrix

In [24]:
sampleSoil = [[2,5,6,6], [5,3,5,5], [6,5,2,5], [6,6,5,2]]

sampleWater = [[6,0,0,0], [0,5,2,0], [0,0,6,0], [0,0,0,6]]

soilInfiltraionMap = {
    'default' : {
        'f0' : 0.01,
        'fc' : 0.02,
        'k' : 0.03
    },
    
    'waterarea' : {
        'f0' : 0,
        'f1' : 0,
        'k' : 0
    },
    
    'settlement' : {
        'f0' : 4.06,
        'fc' : 0.03,
        'k' : 0.45
    },
    
    'rocks' : {
        'f0' : 0.03,
        'fc' : 0.002,
        'k' : 0.02
    }
}

In [11]:
class Cell:
    
    soilType = 'default'
    
    def __init__(self, i, j, soil, water):
        self.i = i
        self.j = j
        self.soil = soil
        self.water = water
        

In [2]:
#get a single 2d array of tuples of water level and soil level
def getSoilLevelPlusWaterLevel(soil, water):
  grid = []
  for i in range(len(soil)):
    row = []
    for j in range(len(soil[i])):
      #row.append((soilLevel[i][j], waterLevel[i][j])) 2d array of tupes
      row.append(Cell(i, j, soil[i][j], water[i][j]))
    grid.append(row)

  return grid

In [3]:
#get cells which has water
def GetWaterCells(grid):
  arr = []
  n = len(grid)
  for i in range(1, n - 1):
    for j in range(1, n - 1):
      if grid[i][j].water > 0:
        arr.append(grid[i][j])
  return arr

In [4]:
def SortWaterCellArray(array):
  return sorted(array, key = lambda i: i.water)

In [5]:
def GetAverageHeight(cell, grid):
  left = grid[cell.i][cell.j + 1].water + grid[cell.i][cell.j + 1].soil 
  top = grid[cell.i + 1][cell.j].water + grid[cell.i + 1][cell.j].soil 
  right = grid[cell.i][cell.j + 1].water + grid[cell.i][cell.j + 1].soil 
  bottom = grid[cell.i - 1][cell.j].water + grid[cell.i - 1][cell.j].soil 
  center = cell.soil + cell.water

  return (left + top + right + bottom + center) / 5

In [6]:
def GetFlowWaterAmount(centerCellHeight, numberOfNeighbours):
  return centerCellHeight / (numberOfNeighbours + 1)

In [7]:
def OverFlowWaterToNeighbour(grid, cell, waterFlowAmount):
  #left
  if(grid[cell.i][cell.j - 1].water + grid[cell.i][cell.j - 1].soil + waterFlowAmount <= cell.water + cell.soil):
    grid[cell.i][cell.j - 1].water += waterFlowAmount
  else:
    grid[cell.i][cell.j - 1].water += (cell.water + cell.soil) - (grid[cell.i][cell.j - 1].water + grid[cell.i][cell.j - 1].soil)
    grid[cell.i][cell.j].water += waterFlowAmount - ((cell.water + cell.soil) - (grid[cell.i][cell.j - 1].water + grid[cell.i][cell.j - 1].soil))

  #top
  if(grid[cell.i - 1][cell.j].water + grid[cell.i - 1][cell.j].soil + waterFlowAmount <= cell.water + cell.soil):
    grid[cell.i - 1][cell.j].water += waterFlowAmount
  else:
    grid[cell.i - 1][cell.j].water += (cell.water + cell.soil) - (grid[cell.i - 1][cell.j].water + grid[cell.i - 1][cell.j].soil)
    grid[cell.i][cell.j].water += waterFlowAmount - ((cell.water + cell.soil) - (grid[cell.i - 1][cell.j].water + grid[cell.i - 1][cell.j].soil))

  #right
  if(grid[cell.i][cell.j + 1].water + grid[cell.i][cell.j + 1].soil + waterFlowAmount <= cell.water + cell.soil):
    grid[cell.i][cell.j + 1].water += waterFlowAmount
  else:
    grid[cell.i][cell.j + 1].water += (cell.water + cell.soil) - (grid[cell.i][cell.j + 1].water + grid[cell.i][cell.j + 1].soil)
    grid[cell.i][cell.j].water += waterFlowAmount - ((cell.water + cell.soil) - (grid[cell.i][cell.j + 1].water + grid[cell.i][cell.j + 1].soil))

  #bottom
  if(grid[cell.i + 1][cell.j].water + grid[cell.i + 1][cell.j].soil + waterFlowAmount <= cell.water + cell.soil):
    grid[cell.i + 1][cell.j].water += waterFlowAmount
  else:
    grid[cell.i + 1][cell.j].water += (cell.water + cell.soil) - (grid[cell.i + 1][cell.j].water + grid[cell.i + 1][cell.j].soil)
    grid[cell.i][cell.j].water += waterFlowAmount - ((cell.water + cell.soil) - (grid[cell.i + 1][cell.j].water + grid[cell.i + 1][cell.j].soil))


In [26]:
def ReduceWaterInfiltration(grid, soilInfiltraionMap, time):
    for i in grid:
        for j in i:
            p = soilInfiltraionMap[j.soilType]['fc']
            q = soilInfiltraionMap[j.soilType]['f0']
            k = soilInfiltraionMap[j.soilType]['k']
            ft = p + ((p - q)*2.78) ** ((-1) * k) * time
            
            ql = ft * time
            
            if(j.water - ql >= 0):
                j.water = j.water - ql
            else:
                j.water = 0

In [ ]:
def Driver():
    for cell in sortedWaterCell:
    averageHeight = GetAverageHeight(cell, grid)
  
    waterFlowAmount = GetFlowWaterAmount(cell.water + cell.soil, 4)

    OverFlowWaterToNeighbour(grid, cell, waterFlowAmount)
    
    ReduceWaterInfiltration(grid, soilInfiltraionMap, 3)

In [27]:
#create grid
grid = getSoilLevelPlusWaterLevel(sampleSoil, sampleWater)

#get cells which has water
waterCells = GetWaterCells(grid)

#sort water cells
sortedWaterCell = SortWaterCellArray(waterCells)[::-1]

#iterate over all water cells
for cell in sortedWaterCell:
  averageHeight = GetAverageHeight(cell, grid)
  
  waterFlowAmount = GetFlowWaterAmount(cell.water + cell.soil, 4)

  OverFlowWaterToNeighbour(grid, cell, waterFlowAmount)
    
  ReduceWaterInfiltration(grid, soilInfiltraionMap, 3)
  

In [ ]:
plt.imshow(matrix)
plt.show()